In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px 
import seaborn as sns
import unidecode, re
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report, roc_auc_score
from scipy import *
from sklearn.feature_selection import SelectKBest, f_regression
from catboost import CatBoostClassifier
import lightgbm as lgb
import joblib
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [2]:
# Basic config
my_font = dict(
            family="Courier New, monospace",
            size=18,
            color="Black",
            variant="small-caps",
) 

In [3]:
df = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")


In [4]:
# df.groupby('SG_UF').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [5]:
# px.histogram(
#     df.groupby(['SG_UF','OBESIDADE']).size().reset_index(name='count').sort_values(by='count', ascending=False),
#     x='SG_UF',
#     y='count',
#     color='SG_UF')

In [6]:

# x = df.groupby('GARGANTA').size().reset_index(name='count').sort_values(by='count', ascending=False)

# px.histogram(
#      x,
#      x='GARGANTA',
#      y='count',
#      color='GARGANTA')
# x

In [7]:
# px.histogram(
#     df.groupby(['SG_UF','CS_SEXO']).size().reset_index(name='count').sort_values(by='count', ascending=False),
#     x='SG_UF',
#     y='count',
#     color='SG_UF')

In [8]:
# X_train = df.select_dtypes(['number']).drop(columns=['EVOLUCAO'])
# y_train = df['EVOLUCAO']

# X_test_sub = df_test.select_dtypes(['number'])

In [9]:
# X_train, X_test, y_train, y_test= train_test_split(X_train, y_train, test_size = .1, random_state=2)


In [10]:
# catb = CatBoostClassifier(
#     iterations=100,      
#     learning_rate=0.1,   
#     depth=6,              
#     verbose=0             
# )

# catb.fit(X_train, y_train)

# preds = catb.predict(X_test)

In [11]:
# catboost_accuracy = accuracy_score(y_test, preds)
# print(f"CatBoost's acurracy: {catboost_accuracy*100:.2f}%")

# print("\nClassification report")
# print(classification_report(y_test, preds))

# cm = confusion_matrix(y_test, preds)
# fig = px.imshow(cm, text_auto=True).update_layout(title={"text": "CatBoost's Confusion Matrix"}, font=my_font).show()


In [12]:
# test_pred = catb.predict_proba(X_test_sub)[:,1] # chances of death, class: 1.0

# pd.DataFrame(np.round(test_pred)).reset_index().to_csv('results/submission_catboost.csv', header=['ID', 'EVOLUCAO'], index=False)

In [13]:
# lightGBM = lgb.LGBMClassifier(random_state=23)

# lightGBM.fit(X_train, y_train)

# preds = lightGBM.predict(X_test)

In [14]:
# accuracy = lightGBM.score(X_test, y_test)


# print(f"LightGBM acurracy: {accuracy*100:.2f}%")
# print("\nClassification report")
# print(classification_report(y_test, preds))

# cm = confusion_matrix(y_test, preds)
# fig = px.imshow(cm, text_auto=True).update_layout(title={"text": "LightGBM's Confusion Matrix"}, font=my_font).show()


In [15]:
# test_pred = lightGBM.predict_proba(X_test_sub)[:,1] # chances of death, class: 1.0

# pd.DataFrame(np.round(test_pred)).reset_index().to_csv('results/submission_light_gbm.csv', header=['ID', 'EVOLUCAO'], index=False)

In [16]:
# rf = RandomForestClassifier()

# rf.fit(X_train, y_train)

In [17]:
# preds = rf.predict(X_test)
# accuracy = rf.score(X_test, y_test)

# print(f"Accuracy: {accuracy*100:.2f}")
# print("\nClassification report")
# print(classification_report(y_test, preds))

# cm = confusion_matrix(y_test, preds)
# fig = px.imshow(cm, text_auto=True).update_layout(title={"text": "Random Forest's Confusion Matrix"}, font=my_font).show()



In [18]:
# xg = XGBClassifier()
# xg.fit(X_train, y_train)
# preds = xg.predict(X_test)


In [19]:
# accuracy = xg.score(X_test, y_test)

# print(f"Accuracy: {accuracy*100:.2f}")
# print("\nClassification report")
# print(classification_report(y_test, preds))

# cm = confusion_matrix(y_test, preds)
# fig = px.imshow(cm, text_auto=True).update_layout(title={"text": "XGBoost's Confusion Matrix"}, font=my_font).show()


In [20]:
# test_pred = xg.predict_proba(X_test_sub)[:,1] # chances of death, class: 1.0

# pd.DataFrame(np.round(test_pred)).reset_index().to_csv('submission_xg.csv', header=['ID', 'EVOLUCAO'], index=False)

# Now we can try to treat this dataset
- Let us start by checking what columns have more missing values
- Then we can drop rows with more that 90% of the data NaN

In [38]:
# px.histogram(df.isna().sum().reset_index(name='count').sort_values(by='count', ascending=False),
#             x='index', y='count', color='count', color_discrete_sequence= px.colors.sequential.Plasma_r,
#              title="Most frequent columns with missing values").update_layout(font = my_font)

In [39]:
# x = df.isna().sum().reset_index(name='count').sort_values(by='count', ascending=False)
# x['pecentage_of_missing_data'] = (x['count'] / len(df))*100
# x

In [23]:
# px.histogram(df.isna().sum().reset_index(name='count').sort_values(by='count', ascending=True).head(20),
#             x='index', y='count', color='count', color_discrete_sequence= px.colors.sequential.Plasma_r,
#              title="Less frequent columns with missing values").update_layout(font = my_font)

In [4]:
missing_counts = df.isnull().sum(axis=1)
thresh = 27
df_no_nulls = df[missing_counts < thresh]


### Note: You cannot just filter out all columns with ANY missing values
 - The accuracy of all models pretty much drops drastically

### Checking correlation between variables

In [41]:
# plt.figure(figsize=(12, 12))
# sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='viridis', fmt=".1f")

## Frist, we can create a total_comorbities columns to represent the total amount of comordities one has.
    - We will separate it in chronic and temporary

In [5]:
df_no_nulls['total_chronic_cormodity'] = ((df_no_nulls[["OBESIDADE", 
                                                        "RENAL", 
                                                        "CARDIOPATI",
                                                        "IMUNODEPRE", 
                                                        "DIABETES", 
                                                        "PNEUMOPATI",
                                                        "HEPATICA",
                                                        "SIND_DOWN"]] == 1.0).sum(axis=1))

df_no_nulls['total_temporary_cormodity'] = (df_no_nulls[["FEBRE",
                                                        "TOSSE",
                                                        "PUERPERA",
                                                        "GARGANTA",
                                                        "DESC_RESP",
                                                        "DIARREIA",
                                                        "VOMITO",
                                                        "FADIGA",
                                                        "SATURACAO",
                                                        "DISPNEIA"]]== 1.0).sum(axis=1)

# Creating a function to handle missing values

In [43]:
df_no_nulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498320 entries, 0 to 498319
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   DT_NOTIFIC                 498320 non-null  object 
 1   CS_SEXO                    498320 non-null  object 
 2   CO_RG_RESI                 425135 non-null  float64
 3   ID_MN_RESI                 498276 non-null  object 
 4   CS_ZONA                    445482 non-null  float64
 5   NU_IDADE_N                 498320 non-null  int64  
 6   CS_ESCOL_N                 351419 non-null  float64
 7   CS_RACA                    478936 non-null  float64
 8   SG_UF                      498276 non-null  object 
 9   CS_GESTANT                 498320 non-null  int64  
 10  PUERPERA                   199555 non-null  float64
 11  DIABETES                   245929 non-null  float64
 12  PNEUMOPATI                 205918 non-null  float64
 13  IMUNODEPRE                 20

In [6]:
class DataPreprocessor:
    def __init__(self, dataframe):
        self.df = dataframe.copy()
    
    def fill_rows_with_ints(self, columns_to_fill, missing_value=0):
        for col in columns_to_fill:
            if col in self.df.columns:
                self.df[col] = self.df[col].fillna(missing_value)
                self.df[col] = self.df[col].astype(int)
        return self.df

    def fill_null_rows(self, columns_to_fill, missing_value):
        for col in columns_to_fill:
            if col in self.df.columns:
                self.df[col] = self.df[col].fillna(missing_value)
                self.df[col] = self.df[col].astype('category')

        return self.df



preprocessor = DataPreprocessor(df_no_nulls)

columns_to_fill = df_no_nulls.select_dtypes(['float']).columns.tolist()

processed_df = preprocessor.fill_rows_with_ints( columns_to_fill, missing_value=9)


In [7]:
columns_to_fill = [
                   "OBESIDADE", 
                   "RENAL", 
                   "CARDIOPATI",
                   "IMUNODEPRE", 
                   "DIABETES", 
                   "PNEUMOPATI",
                   "HEPATICA",
                   "FEBRE",
                   "TOSSE",
                   "GARGANTA",
                   "DESC_RESP",
                   "DIARREIA",
                   "VOMITO",
                   "FADIGA",
                   "SATURACAO",
                   "DISPNEIA",
                   "CS_RACA",
                   "CS_ZONA",
                   "VACINA",
                   "CS_ESCOL_N",
                   "FATOR_RISC",
                   "SIND_DOWN",
                   "CO_RG_RESI",
                   "PUERPERA",
                   "CS_SEXO",
                   "SG_UF",
                   "EVOLUCAO",
                   "CS_GESTANT"
                  ] # 22 columns 

preprocessor = DataPreprocessor(processed_df)

df_2 = preprocessor.fill_null_rows(columns_to_fill, 9)


In [9]:
df_2.drop(columns={"DT_NOTIFIC", "ID_MN_RESI","OBES_IMC", "CO_RG_RESI"}, inplace=True)

In [10]:
X = df_2.drop(columns={'EVOLUCAO'})
y = df_2['EVOLUCAO']

In [11]:

X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = .2, random_state=32)


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=32)

cat_features = X.select_dtypes(['category']).columns.tolist()
params = {
    'iterations':1000,
    'learning_rate':0.05,
    'depth':6,
    'eval_metric':'TotalF1',
    'random_seed':42,
    'early_stopping_rounds':20,
    'verbose': 50
}
ctb = CatBoostClassifier(
  **params
)

ctb.fit(
    X_train, y_train,
    eval_set=(X_valid, y_valid),
    cat_features=cat_features
)

y_pred = ctb.predict_proba(X_valid)[:, 1]
print("Validation AUC:", roc_auc_score(y_valid, y_pred))

0:	learn: 0.6909215	test: 0.6924677	best: 0.6924677 (0)	total: 713ms	remaining: 11m 52s
50:	learn: 0.7175420	test: 0.7199444	best: 0.7199444 (50)	total: 28.2s	remaining: 8m 44s
100:	learn: 0.7250153	test: 0.7276467	best: 0.7276825 (99)	total: 56.8s	remaining: 8m 25s
150:	learn: 0.7290588	test: 0.7313282	best: 0.7313282 (150)	total: 1m 26s	remaining: 8m 7s
200:	learn: 0.7311059	test: 0.7322920	best: 0.7322991 (199)	total: 2m	remaining: 7m 57s
250:	learn: 0.7326083	test: 0.7336522	best: 0.7337329 (247)	total: 2m 33s	remaining: 7m 37s
300:	learn: 0.7339184	test: 0.7349994	best: 0.7350185 (298)	total: 3m 6s	remaining: 7m 12s
350:	learn: 0.7349216	test: 0.7356365	best: 0.7358116 (343)	total: 3m 37s	remaining: 6m 42s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.7358116128
bestIteration = 343

Shrink model to first 344 iterations.
Validation AUC: 0.7997157099675185


In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# Define the parameter grid to search
param_grid = {
    'iterations': [100, 200, 300],
    'depth': [6, 8, 10],
    'learning_rate': [0.1, 0.05, 0.01]
}

# Initialize the CatBoost model
catboost_model = CatBoostClassifier(cat_features=cat_features, verbose=False)

# Initialize GridSearchCV with the model, parameter grid, and cross-validation strategy
grid_search = GridSearchCV(
    estimator=catboost_model,
    param_grid=param_grid,
    scoring='f1',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1,  # Use all available CPU cores
    verbose=1
)

# Fit the grid search to the data (this will run all the cross-validation folds)
grid_search.fit(X, y)

# Get the best hyperparameters and score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [ ]:
preds = ctb.predict(X_test)

accuracy = ctb.score(X_test, y_test)


print(f"ctb acurracy: {accuracy*100:.2f}%")
print("\nClassification report")
print(classification_report(y_test, preds))

cm = confusion_matrix(y_test, preds)
fig = px.imshow(cm, text_auto=True).update_layout(title={"text": "CatBoost's Confusion Matrix"}, font=my_font).show()













from catboost import Pool, cv
train_pool = Pool(X, label=y, cat_features=cat_features)

params = {
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'eval_metric': 'TotalF1',
    'random_seed': 42,
    'loss_function': 'Logloss',

}
cv_results = cv(
    params=params,
    pool=train_pool,
    fold_count=1,
    early_stopping_rounds=50,
    verbose=50,
    stratified=True
)

print("Best CV AUC:", cv_results['test-AUC-mean'].max())


In [ ]:
df_test['total_chronic_cormodity'] = ((df_test[["OBESIDADE", 
                                                        "RENAL", 
                                                        "CARDIOPATI",
                                                        "IMUNODEPRE", 
                                                        "DIABETES", 
                                                        "PNEUMOPATI",
                                                        "HEPATICA",
                                                        "SIND_DOWN"]] == 1.0).sum(axis=1))

df_test['total_temporary_cormodity'] = (df_test[[       "FEBRE",
                                                        "TOSSE",
                                                        "PUERPERA",
                                                        "GARGANTA",
                                                        "DESC_RESP",
                                                        "DIARREIA",
                                                        "VOMITO",
                                                        "FADIGA",
                                                        "SATURACAO",
                                                        "DISPNEIA"]]== 1.0).sum(axis=1)
columns_to_transform = df_test_copy.columns.tolist()
columns_to_drop = ["NU_IDADE_N", "total_chronic_cormodity", "total_temporary_cormodity"]

columns_to_drop_filtered = [item for item in  columns_to_transform if item not in columns_to_drop ]


columns_to_drop_filtered


def to_categorical (columns, df):
    for col in columns:
            if col in df.columns:
                df[col] = df[col].astype('category')
    return df


df_test_copy = to_categorical(columns_to_drop_filtered, df_test_copy)


In [ ]:
test_pred = ctb.predict_proba(df_test_copy)[:,1]
test_pred

In [ ]:
# pd.DataFrame(np.round(test_pred)).reset_index().to_csv('lightgbm_tuned_not_coded.csv', header=['ID', 'EVOLUCAO'], index=False)